![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2Farchitectures%2Ftracking%2Fsetup%2Fpixel&file=pixel-setup-reporting.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/pixel/pixel-setup-reporting.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2Farchitectures%2Ftracking%2Fsetup%2Fpixel%2Fpixel-setup-reporting.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/pixel/pixel-setup-reporting.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/architectures/tracking/setup/pixel/pixel-setup-reporting.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Setup Reporting: `pixel_daily_loads`

This workflor represents the intial setup work and does not need to be repeated.  See the Pixel Reporting notebook in the same folder for the ongoing inserts of new data.

Initial Setup:
- review the source tables, new and old
- create `pixel_daily_loads`
- Initial load of the original GA4 data
- Initial load of the new custom tracking data collected so far
- Identify the gap
- Forcast with original GA4 data
- Load forecasting data to `pixel_daily_loads`
- Copy forecasted values to actual for the missing days


## Source Tables

In [2]:
%%bigquery
SELECT *
FROM `vertex-ai-mlops-369716.reporting.daily_loads`
WHERE event_date = '2024-01-01'
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,file,loads,row_type,lower_bound,upper_bound
0,2024-01-01,statmike/vertex-ai-mlops/IDE/readme.md,2.0,actual,NaN,NaN
1,2024-01-01,statmike/vertex-ai-mlops/scripting/readme.md,3.0,actual,NaN,NaN
2,2024-01-01,statmike/vertex-ai-mlops/02 - Vertex AI AutoML...,4.0,actual,NaN,NaN
3,2024-01-01,statmike/vertex-ai-mlops/03 - BigQuery ML (BQM...,4.0,actual,NaN,NaN
4,2024-01-01,statmike/vertex-ai-mlops/03 - BigQuery ML (BQM...,4.0,actual,NaN,NaN


In [5]:
%%bigquery
SELECT *
FROM `vertex-ai-mlops-369716.analytics_343629755.events_intraday_20240101`
LIMIT 2

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,collected_traffic_source,is_active_user
0,20240101,1704087968397161,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,-435832983,<NA>,None,1,...,"{'city': '', 'country': '', 'continent': '(not...",None,None,4308611918,WEB,None,None,[],None,<NA>
1,20240101,1704093423757679,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,724560239,<NA>,None,1,...,"{'city': '', 'country': '', 'continent': '(not...",None,None,4308611918,WEB,None,None,[],None,<NA>


In [4]:
%%bigquery
SELECT *
FROM `vertex-ai-mlops-369716.pixel_tracking.pixel-tracking-data`
WHERE DATE(event_timestamp) = '2024-04-01'
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_timestamp,file_path,file_name,client,source
0,2024-04-01 14:22:23.905150+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom
1,2024-04-01 11:50:42.983237+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom
2,2024-04-01 13:46:25.655135+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom
3,2024-04-01 14:02:14.287003+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom
4,2024-04-01 12:29:40.482654+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom


## Create Table: `pixel_daily_loads`

PARTITION BY: event_date

SCHEMA:
- event_date
- file_path
- file_name
- data_source: ga4, pixel, forecast
- loads: count
- row_type: actual/forecast
- lower_bound
- upper_bound

In [6]:
%%bigquery
CREATE TABLE IF NOT EXISTS `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
  (
    event_date DATE,
    file_path STRING,
    file_name STRING,
    loads FLOAT64,
    data_source STRING,
    row_type STRING,
    lower_bound FLOAT64,
    upper_bound FLOAT64
  )
  PARTITION BY event_date

Query is running:   0%|          |

""


## Initial Load Of GA4 Data

Extract the date from the timestamp after converting the timestamp to EST time - default is UTC so -05.

### Review A Single Day

In [12]:
%%bigquery
WITH DAILY AS (
  SELECT
    EXTRACT(DATE FROM TIMESTAMP_MICROS(event_timestamp) AT TIME ZONE '-05') as event_date,
    (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_location') AS file_path,
    (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_title') AS file_name,
    COUNT(*) AS loads
  FROM `vertex-ai-mlops-369716.analytics_343629755.events_intraday_20240101`
  GROUP BY event_date, file_path, file_name
)
SELECT *,
  'GA4' AS data_source,
  'actual' AS row_type,
  NULL AS lower_bound,
  NULL AS upper_bound
FROM DAILY

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,file_path,file_name,loads,data_source,row_type,lower_bound,upper_bound
0,2024-01-01,statmike/vertex-ai-mlops,readme.md,67,GA4,actual,<NA>,<NA>
1,2024-01-01,statmike/vertex-ai-mlops/00 - Setup,readme.md,7,GA4,actual,<NA>,<NA>
2,2024-01-01,statmike/vertex-ai-mlops/01 - Data Sources,readme.md,1,GA4,actual,<NA>,<NA>
3,2024-01-01,statmike/vertex-ai-mlops/scripting,readme.md,3,GA4,actual,<NA>,<NA>
4,2024-01-01,statmike/vertex-ai-mlops/02 - Vertex AI AutoML,readme.md,5,GA4,actual,<NA>,<NA>
5,2024-01-01,statmike/vertex-ai-mlops/Applied Forecasting,readme.md,1,GA4,actual,<NA>,<NA>
6,2024-01-01,statmike/vertex-ai-mlops/00 - Setup,00 - Environment Setup.ipynb,7,GA4,actual,<NA>,<NA>
7,2024-01-01,statmike/vertex-ai-mlops/02 - Vertex AI AutoML,02a - Vertex AI - AutoML in GCP Console (no co...,4,GA4,actual,<NA>,<NA>
8,2024-01-01,statmike/vertex-ai-mlops/03 - BigQuery ML (BQML),03c - BQML Random Forest.ipynb,4,GA4,actual,<NA>,<NA>
9,2024-01-01,statmike/vertex-ai-mlops/03 - BigQuery ML (BQML),03Tools - Pipelines Example 2.ipynb,4,GA4,actual,<NA>,<NA>


### Load All Days

In [13]:
%%bigquery
INSERT INTO `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
  WITH DAILY AS (
    SELECT
      EXTRACT(DATE FROM TIMESTAMP_MICROS(event_timestamp) AT TIME ZONE '-05') as event_date,
      (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_location') AS file_path,
      (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_title') AS file_name,
      COUNT(*) AS loads
    FROM `vertex-ai-mlops-369716.analytics_343629755.events_intraday_*`
    GROUP BY event_date, file_path, file_name
  )
  SELECT *,
    'GA4' AS data_source,
    'actual' AS row_type,
    NULL AS lower_bound,
    NULL AS upper_bound
  FROM DAILY

Query is running:   0%|          |

""


## Load Initial Pixel Tracking Data

### Review A Single Day

In [15]:
%%bigquery
WITH DAILY AS (
  SELECT
    EXTRACT(DATE FROM event_timestamp AT TIME ZONE '-05') AS event_date,
    file_path, file_name, COUNT(*) AS loads
  FROM `vertex-ai-mlops-369716.pixel_tracking.pixel-tracking-data`
  WHERE DATE(event_timestamp) = '2024-04-01'
  GROUP BY event_date, file_path, file_name
)
SELECT *,
  'pixel' AS data_source,
  'actual' AS row_type,
  NULL AS lower_bound,
  NULL AS upper_bound
FROM DAILY

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,file_path,file_name,loads,data_source,row_type,lower_bound,upper_bound
0,2024-04-01,statmike/vertex-ai-mlops,readme.md,37,pixel,actual,<NA>,<NA>
1,2024-03-31,statmike/vertex-ai-mlops,readme.md,2,pixel,actual,<NA>,<NA>
2,2024-04-01,statmike/vertex-ai-mlops/Dev,Notes - Just Notes.ipynb,1,pixel,actual,<NA>,<NA>
3,2024-04-01,statmike/vertex-ai-mlops/MLOps,Vertex AI Pipelines - Control.ipynb,1,pixel,actual,<NA>,<NA>
4,2024-04-01,statmike/vertex-ai-mlops/Dev/new,Geospatial Indexing.ipynb,2,pixel,actual,<NA>,<NA>
...,...,...,...,...,...,...,...,...
67,2024-04-01,statmike/vertex-ai-mlops/architectures/trackin...,prod-tracking-pixel.ipynb,1,pixel,actual,<NA>,<NA>
68,2024-04-01,statmike/vertex-ai-mlops/Applied GenAI,Vertex AI GenAI For Document Q&A - FAA Regulat...,1,pixel,actual,<NA>,<NA>
69,2024-04-01,statmike/vertex-ai-mlops/04 - scikit-learn,04c - Vertex AI Custom Model - scikit-learn - ...,2,pixel,actual,<NA>,<NA>
70,2024-04-01,statmike/vertex-ai-mlops/04 - scikit-learn,04d - Vertex AI Custom Model - scikit-learn - ...,1,pixel,actual,<NA>,<NA>


### Load All Days

In [16]:
%%bigquery
INSERT INTO `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
  WITH DAILY AS (
    SELECT
      EXTRACT(DATE FROM event_timestamp AT TIME ZONE '-05') AS event_date,
      file_path, file_name, COUNT(*) AS loads
    FROM `vertex-ai-mlops-369716.pixel_tracking.pixel-tracking-data`
    GROUP BY event_date, file_path, file_name
  )
  SELECT *,
    'pixel' AS data_source,
    'actual' AS row_type,
    NULL AS lower_bound,
    NULL AS upper_bound
  FROM DAILY

Query is running:   0%|          |

""


## Identify The Gap

Missing data where GA4 ended and before the custom pixel tracking was active in production.  Remember to clip the boundary days.

In [24]:
%%bigquery
SELECT GA4_END, PIXEL_BEGIN, DATE_DIFF(PIXEL_BEGIN, GA4_END, DAY) MISSING_DAYS FROM
(SELECT MAX(event_date) AS GA4_END
FROM `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
WHERE data_source = 'GA4')
CROSS JOIN
(SELECT MIN(event_date) AS PIXEL_BEGIN
FROM `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
WHERE data_source = 'pixel')

Query is running:   0%|          |

Downloading:   0%|          |

,GA4_END,PIXEL_BEGIN,MISSING_DAYS
0,2024-03-12,2024-03-29,17


## Forecasting With GA4 - Document Level

Use this to cover double the missing period as horizon.

Use hierarchical forecasting to get document level forecast.

In [69]:
%%bigquery
CREATE OR REPLACE MODEL `vertex-ai-mlops-369716.reporting.pixel_daily_loads_forecast`
OPTIONS
  (model_type = 'ARIMA_PLUS',
    time_series_timestamp_col = 'event_date',
    time_series_data_col = 'loads',
    TIME_SERIES_ID_COL = ['file_path', 'file_name'],
    data_frequency = 'DAILY',
    auto_arima_max_order = 5,
    #holiday_region = 'US',
    horizon = 200,
    #FORECAST_LIMIT_LOWER_BOUND = 0,
    HIERARCHICAL_TIME_SERIES_COLS = ['file_path', 'file_name']
  ) AS
SELECT event_date, file_path, file_name, loads
FROM `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
WHERE data_source = 'GA4' AND row_type = 'actual'

Query is running:   0%|          |

""


### Review a History Day

In [75]:
%%bigquery history
SELECT *
FROM ML.EXPLAIN_FORECAST(
    MODEL `vertex-ai-mlops-369716.reporting.pixel_daily_loads_forecast`,
    STRUCT(200 AS horizon, 0.95 AS confidence_level))
WHERE DATE(time_series_timestamp) = '2024-03-12'
#ORDER BY event_date

Query is running:   0%|          |

Downloading:   0%|          |

In [76]:
history

,file_path,file_name,time_series_timestamp,time_series_type,time_series_data,time_series_adjusted_data,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,trend,seasonal_period_yearly,seasonal_period_quarterly,seasonal_period_monthly,seasonal_period_weekly,seasonal_period_daily,holiday_effect,spikes_and_dips,step_changes,residual
0,GoogleCloudPlatform/bigquery_ml_utils/notebooks,None,2024-03-12 00:00:00+00:00,forecast,-0.158048,-0.158048,4.165943,0.95,-8.308542,7.992446,0.994255,NaN,NaN,NaN,-1.152303,NaN,NaN,NaN,NaN,NaN
1,GoogleCloudPlatform/bigquery_ml_utils/notebooks,bqml-feature-engineering.ipynb,2024-03-12 00:00:00+00:00,forecast,-0.158048,-0.158048,4.165943,0.95,-8.308542,7.992446,0.994255,NaN,NaN,NaN,-1.152303,NaN,NaN,NaN,NaN,NaN
2,statmike/vertex-ai-mlops,11 - Vertex AI > Features - Feature Store.ipynb,2024-03-12 00:00:00+00:00,forecast,1.242854,1.242854,1.923538,0.95,-2.520468,5.006176,1.221284,NaN,NaN,NaN,0.021570,NaN,NaN,NaN,NaN,NaN
3,statmike/vertex-ai-mlops,MLOps.md,2024-03-12 00:00:00+00:00,forecast,3.448296,3.448296,0.547908,0.95,2.376337,4.520254,3.305687,NaN,NaN,NaN,0.142608,NaN,NaN,NaN,NaN,NaN
4,statmike/vertex-ai-mlops,readme.md,2024-03-12 00:00:00+00:00,history,89.000000,106.799869,27.762786,NaN,NaN,NaN,79.696709,NaN,NaN,NaN,27.103160,NaN,NaN,0.0,0.0,-17.799869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,statmike/vertex-ai-mlops/architectures/trackin...,readme.md,2024-03-12 00:00:00+00:00,forecast,1.000000,1.000000,3.510815,0.95,-5.868763,7.868763,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,statmike/vertex-ai-mlops/architectures/trackin...,None,2024-03-12 00:00:00+00:00,forecast,1.000000,1.000000,9.573925,0.95,-17.730983,19.730983,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,statmike/vertex-ai-mlops/architectures/trackin...,readme.md,2024-03-12 00:00:00+00:00,forecast,1.000000,1.000000,9.573925,0.95,-17.730983,19.730983,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,statmike/vertex-ai-mlops/scripting,None,2024-03-12 00:00:00+00:00,history,1.000000,1.056185,0.284390,NaN,NaN,NaN,1.106249,NaN,NaN,NaN,-0.050065,NaN,NaN,0.0,NaN,-0.056185


### Review a Horizon Day

In [77]:
%%bigquery horizon
SELECT *
FROM ML.EXPLAIN_FORECAST(
    MODEL `vertex-ai-mlops-369716.reporting.pixel_daily_loads_forecast`,
    STRUCT(200 AS horizon, 0.95 AS confidence_level))
WHERE time_series_type = 'forecast' and DATE(time_series_timestamp) = '2024-03-13'
#ORDER BY event_date

Query is running:   0%|          |

Downloading:   0%|          |

In [78]:
horizon

,file_path,file_name,time_series_timestamp,time_series_type,time_series_data,time_series_adjusted_data,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,trend,seasonal_period_yearly,seasonal_period_quarterly,seasonal_period_monthly,seasonal_period_weekly,seasonal_period_daily,holiday_effect,spikes_and_dips,step_changes,residual
0,GoogleCloudPlatform/bigquery_ml_utils/notebooks,None,2024-03-13 00:00:00+00:00,forecast,1.038854,1.038854,4.172689,0.95,-7.124837,9.202544,0.994292,NaN,NaN,NaN,0.044562,NaN,NaN,NaN,NaN,NaN
1,GoogleCloudPlatform/bigquery_ml_utils/notebooks,bqml-feature-engineering.ipynb,2024-03-13 00:00:00+00:00,forecast,1.038854,1.038854,4.172689,0.95,-7.124837,9.202544,0.994292,NaN,NaN,NaN,0.044562,NaN,NaN,NaN,NaN,NaN
2,statmike/vertex-ai-mlops,11 - Vertex AI > Features - Feature Store.ipynb,2024-03-13 00:00:00+00:00,forecast,1.176151,1.176151,1.952495,0.95,-2.643825,4.996126,1.223246,NaN,NaN,NaN,-0.047095,NaN,NaN,NaN,NaN,NaN
3,statmike/vertex-ai-mlops,MLOps.md,2024-03-13 00:00:00+00:00,forecast,3.437062,3.437062,0.651139,0.95,2.163135,4.710988,3.305687,NaN,NaN,NaN,0.131375,NaN,NaN,NaN,NaN,NaN
4,statmike/vertex-ai-mlops,readme.md,2024-03-13 00:00:00+00:00,forecast,93.106465,93.106465,27.762786,0.95,38.789738,147.423193,78.923557,NaN,NaN,NaN,14.182908,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,statmike/vertex-ai-mlops/architectures/trackin...,readme.md,2024-03-13 00:00:00+00:00,forecast,1.000000,1.000000,3.528682,0.95,-5.903718,7.903718,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,statmike/vertex-ai-mlops/architectures/trackin...,None,2024-03-13 00:00:00+00:00,forecast,1.000000,1.000000,9.720093,0.95,-18.016954,20.016954,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,statmike/vertex-ai-mlops/architectures/trackin...,readme.md,2024-03-13 00:00:00+00:00,forecast,1.000000,1.000000,9.720093,0.95,-18.016954,20.016954,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,statmike/vertex-ai-mlops/scripting,None,2024-03-13 00:00:00+00:00,forecast,1.156212,1.156212,0.284967,0.95,0.598686,1.713739,1.153831,NaN,NaN,NaN,0.002381,NaN,NaN,NaN,NaN,NaN


In [80]:
horizon[horizon['file_path']=='statmike/vertex-ai-mlops']

,file_path,file_name,time_series_timestamp,time_series_type,time_series_data,time_series_adjusted_data,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,trend,seasonal_period_yearly,seasonal_period_quarterly,seasonal_period_monthly,seasonal_period_weekly,seasonal_period_daily,holiday_effect,spikes_and_dips,step_changes,residual
2,statmike/vertex-ai-mlops,11 - Vertex AI > Features - Feature Store.ipynb,2024-03-13 00:00:00+00:00,forecast,1.176151,1.176151,1.952495,0.95,-2.643825,4.996126,1.223246,NaN,NaN,NaN,-0.047095,NaN,NaN,NaN,NaN,NaN
3,statmike/vertex-ai-mlops,MLOps.md,2024-03-13 00:00:00+00:00,forecast,3.437062,3.437062,0.651139,0.95,2.163135,4.710988,3.305687,NaN,NaN,NaN,0.131375,NaN,NaN,NaN,NaN,NaN
4,statmike/vertex-ai-mlops,readme.md,2024-03-13 00:00:00+00:00,forecast,93.106465,93.106465,27.762786,0.95,38.789738,147.423193,78.923557,NaN,NaN,NaN,14.182908,NaN,NaN,NaN,NaN,NaN


In [82]:
horizon[(horizon['file_path'].isnull() & horizon['file_name'].isnull())]

,file_path,file_name,time_series_timestamp,time_series_type,time_series_data,time_series_adjusted_data,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,trend,seasonal_period_yearly,seasonal_period_quarterly,seasonal_period_monthly,seasonal_period_weekly,seasonal_period_daily,holiday_effect,spikes_and_dips,step_changes,residual


In [74]:
check[(check['file_path'].notnull() & check['file_name'].notnull())]

,file_path,file_name,time_series_timestamp,time_series_type,time_series_data,time_series_adjusted_data,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,trend,seasonal_period_yearly,seasonal_period_quarterly,seasonal_period_monthly,seasonal_period_weekly,seasonal_period_daily,holiday_effect,spikes_and_dips,step_changes,residual
1,GoogleCloudPlatform/bigquery_ml_utils/notebooks,bqml-feature-engineering.ipynb,2024-03-13 00:00:00+00:00,forecast,1.038854,1.038854,4.172689,0.95,-7.124837,9.202544,0.994292,NaN,NaN,NaN,0.044562,NaN,NaN,NaN,NaN,NaN
2,statmike/vertex-ai-mlops,11 - Vertex AI > Features - Feature Store.ipynb,2024-03-13 00:00:00+00:00,forecast,1.176151,1.176151,1.952495,0.95,-2.643825,4.996126,1.223246,NaN,NaN,NaN,-0.047095,NaN,NaN,NaN,NaN,NaN
3,statmike/vertex-ai-mlops,MLOps.md,2024-03-13 00:00:00+00:00,forecast,3.437062,3.437062,0.651139,0.95,2.163135,4.710988,3.305687,NaN,NaN,NaN,0.131375,NaN,NaN,NaN,NaN,NaN
4,statmike/vertex-ai-mlops,readme.md,2024-03-13 00:00:00+00:00,forecast,93.106465,93.106465,27.762786,0.95,38.789738,147.423193,78.923557,NaN,NaN,NaN,14.182908,NaN,NaN,NaN,NaN,NaN
6,statmike/vertex-ai-mlops/00 - Setup,00 - Environment Setup.ipynb,2024-03-13 00:00:00+00:00,forecast,11.422495,11.422495,4.958443,0.95,1.721508,21.123481,10.147377,NaN,NaN,NaN,1.275118,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,statmike/vertex-ai-mlops/architectures/trackin...,GitHub Metrics - 3 - Traffic - Incremental Upd...,2024-03-13 00:00:00+00:00,forecast,1.066282,1.066282,1.232924,0.95,-1.345882,3.478447,1.066282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,statmike/vertex-ai-mlops/architectures/trackin...,readme.md,2024-03-13 00:00:00+00:00,forecast,-2.955805,-2.955805,10.218191,0.95,-22.947268,17.035658,-2.975031,NaN,NaN,NaN,0.019226,NaN,NaN,NaN,NaN,NaN
335,statmike/vertex-ai-mlops/architectures/trackin...,readme.md,2024-03-13 00:00:00+00:00,forecast,1.000000,1.000000,3.528682,0.95,-5.903718,7.903718,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,statmike/vertex-ai-mlops/architectures/trackin...,readme.md,2024-03-13 00:00:00+00:00,forecast,1.000000,1.000000,9.720093,0.95,-18.016954,20.016954,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Forecasting With GA4 - Overall Only



In [101]:
%%bigquery
CREATE OR REPLACE MODEL `vertex-ai-mlops-369716.reporting.pixel_daily_loads_forecast`
OPTIONS
  (model_type = 'ARIMA_PLUS',
    time_series_timestamp_col = 'event_date',
    time_series_data_col = 'loads',
    data_frequency = 'DAILY',
    auto_arima_max_order = 5,
    holiday_region = 'US',
    horizon = 60#,
    #FORECAST_LIMIT_LOWER_BOUND = 0
  ) AS
SELECT event_date, SUM(loads) as loads
FROM `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
WHERE data_source = 'GA4' AND row_type = 'actual' AND event_date <= '2024-03-11'
GROUP BY event_date

Query is running:   0%|          |

""


In [104]:
%%bigquery
SELECT
  DATE(time_series_timestamp) AS event_date,
  Null as file_path,
  Null as file_name,
  time_series_adjusted_data AS loads,
  'forecast' AS data_source,
  'forecast' AS row_type,
  prediction_interval_lower_bound AS lower_bound,
  prediction_interval_upper_bound AS upper_bound
FROM ML.EXPLAIN_FORECAST(
    MODEL `vertex-ai-mlops-369716.reporting.pixel_daily_loads_forecast`,
    STRUCT(60 AS horizon, 0.95 AS confidence_level))
WHERE DATE(time_series_timestamp) <= '2024-04-21'
ORDER BY event_date

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,file_path,file_name,loads,data_source,row_type,lower_bound,upper_bound
0,2022-11-27,<NA>,<NA>,62.000000,forecast,forecast,NaN,NaN
1,2022-11-28,<NA>,<NA>,261.771702,forecast,forecast,NaN,NaN
2,2022-11-29,<NA>,<NA>,340.351851,forecast,forecast,NaN,NaN
3,2022-11-30,<NA>,<NA>,257.699843,forecast,forecast,NaN,NaN
4,2022-12-01,<NA>,<NA>,258.181501,forecast,forecast,NaN,NaN
...,...,...,...,...,...,...,...,...
507,2024-04-17,<NA>,<NA>,370.550168,forecast,forecast,92.168834,648.931501
508,2024-04-18,<NA>,<NA>,342.199089,forecast,forecast,62.874896,621.523283
509,2024-04-19,<NA>,<NA>,409.312639,forecast,forecast,129.048757,689.576521
510,2024-04-20,<NA>,<NA>,277.859999,forecast,forecast,-3.340431,559.060428


## Load Initial Forecast Data

In [113]:
%%bigquery
INSERT INTO `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
  SELECT
    DATE(time_series_timestamp) AS event_date,
    Null as file_path,
    Null as file_name,
    time_series_adjusted_data AS loads,
    'forecast' AS data_source,
    'forecast' AS row_type,
    prediction_interval_lower_bound AS lower_bound,
    prediction_interval_upper_bound AS upper_bound
  FROM ML.EXPLAIN_FORECAST(
      MODEL `vertex-ai-mlops-369716.reporting.pixel_daily_loads_forecast`,
      STRUCT(60 AS horizon, 0.95 AS confidence_level))
  WHERE DATE(time_series_timestamp) <= '2024-04-21'

Query is running:   0%|          |

""


## Copy Forecast To Actual For Missing Zone Between GA4 And Custom

In [109]:
%%bigquery
SELECT
  DATE(time_series_timestamp) AS event_date,
  Null as file_path,
  Null as file_name,
  ABS(time_series_adjusted_data) AS loads,
  'forecast' AS data_source,
  'actual' AS row_type,
  prediction_interval_lower_bound AS lower_bound,
  prediction_interval_upper_bound AS upper_bound
FROM ML.EXPLAIN_FORECAST(
    MODEL `vertex-ai-mlops-369716.reporting.pixel_daily_loads_forecast`,
    STRUCT(60 AS horizon, 0.95 AS confidence_level))
WHERE
  DATE(time_series_timestamp) > '2024-03-12'
  AND
  DATE(time_series_timestamp) < '2024-03-29'
ORDER BY event_date

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,file_path,file_name,loads,data_source,row_type,lower_bound,upper_bound
0,2024-03-13,<NA>,<NA>,414.793354,forecast,actual,238.734256,590.852452
1,2024-03-14,<NA>,<NA>,394.065550,forecast,actual,217.961673,570.169428
2,2024-03-15,<NA>,<NA>,441.647179,forecast,actual,234.905278,648.389080
3,2024-03-16,<NA>,<NA>,93.926353,forecast,actual,-152.385120,340.237826
4,2024-03-17,<NA>,<NA>,119.571785,forecast,actual,-366.948381,127.804810
5,2024-03-18,<NA>,<NA>,353.754438,forecast,actual,105.317287,602.191590
6,2024-03-19,<NA>,<NA>,427.239116,forecast,actual,177.745917,676.732316
7,2024-03-20,<NA>,<NA>,369.916424,forecast,actual,119.371627,620.461220
8,2024-03-21,<NA>,<NA>,346.579553,forecast,actual,94.987555,598.171550
9,2024-03-22,<NA>,<NA>,400.360139,forecast,actual,147.725281,652.994997


In [114]:
%%bigquery
INSERT INTO `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
  SELECT
    DATE(time_series_timestamp) AS event_date,
    '' as file_path,
    '' as file_name,
    ABS(time_series_adjusted_data) AS loads,
    'forecast' AS data_source,
    'actual' AS row_type,
    prediction_interval_lower_bound AS lower_bound,
    prediction_interval_upper_bound AS upper_bound
  FROM ML.EXPLAIN_FORECAST(
      MODEL `vertex-ai-mlops-369716.reporting.pixel_daily_loads_forecast`,
      STRUCT(60 AS horizon, 0.95 AS confidence_level))
  WHERE
    DATE(time_series_timestamp) > '2024-03-12'
    AND
    DATE(time_series_timestamp) < '2024-03-29'
  ORDER BY event_date

Query is running:   0%|          |

""


In [112]:
#%%bigquery
#DELETE
#FROM `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
#WHERE data_source = 'forecast';

Query is running:   0%|          |

""
